<img src="./src/logo.png" width="250">

**Baustein:** Daten  $\rightarrow$ **Subbaustein:** Datenvorverarbeitung $\rightarrow$ **Übungsserie**

**Version:** 2.0, **Lizenz:** <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND 4.0</a>

***

# Daten: Datenvorverarbeitung

---
## Importieren der notwendigen Python-Bibliotheken

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

%run src/setup.ipynb

---
## Importieren der Daten
### Aufgabe 1: Passen Sie den Importierbefehl so an, dass der gewünschte Datensatz *studierende.csv* in der Variable ```df``` gespeichert ist.
Hierbei hilft die Bibliothek `pandas`, die es erlaubt, zum Beispiel eine `.csv`-Datei als **Dataframe-Objekt** einzulesen.

In [ ]:
df = pd.read_csv('./data/studierende.csv') # Laden des Datensatzes

---
## Datensatz
### Aufgabe 2: Machen Sie sich mit dem Datensatz vertraut und notieren Sie potenzielle Probleme.
Nutzen Sie dafür die `pandas`-Funktionen, die Sie in der *Daten: Deskriptive Statistik und Visualisierung*-Übung kennengelernt haben.

In [ ]:
df.head()

In [ ]:
df.info()

Antwort:

### Aufgabe 3: Beantworten Sie die nachfolgenden Fragen zum Datensatz.

In [ ]:
%run src/01_Fragen_DAT_PJY_002.ipynb

---
## Duplikate und Leerstellen
Wir werden nun verschiedene Methoden für den Umgang mit fehlenden/doppelten Werten betrachten. Da es sich um einen Zufallsdatensatz handelt, soll damit gezeigt werden, wie wir mit fehlenden/doppelten Werten umgehen können. Bei echten Daten müssen wir natürlich die Daten sorgfältig untersuchen und eine fundierte Entscheidung treffen.

Um Zeilen zu finden, die fehlende Werte enthalten nutzen wir die untenstehende Code-Zeile(n). Wir wollen alle Zeilen im Datensatz finden und anzeigen, in denen mindestens ein Wert fehlt.

In [ ]:
df[df.isna().any(axis=1)]

Zur Anschaulichkeit werden die Befehle nun einzeln betrachtet, um den Code besser zu verstehen.

1. `df.isna()`: Diese Methode wird auf unseren Dataframe `df` angewendet und gibt einen DataFrame gleicher Größe zurück, der `True` an den Positionen enthält, an denen die ursprünglichen Daten `NaN` (Not a Number) oder fehlende Werte sind, und `False` an allen anderen Positionen.

In [ ]:
df.isna()

2. `.any(axis=1)`: Diese Methode wird auf den booleschen Dataframe angewendet, der von `df.isna()` zurückgegeben wird. Der Parameter `axis=1` bedeutet, dass die Methode entlang der Zeilen arbeitet. `.any(axis=1)` gibt eine Pandas Series zurück, die `True` für jede Zeile enthält, die mindestens einen `True`-Wert hat (d.h. mindestens einen fehlenden Wert in der ursprünglichen Zeile des Dataframes).

In [ ]:
df.isna().any(axis=1)

3. `df[...]`: Schließlich wird der ursprüngliche Dataframe `df` mithilfe dieser booleschen Series gefiltert. Das bedeutet, dass nur die Zeilen von `df` zurückgegeben werden, für die der Wert in der booleschen Series `True` ist.

In [ ]:
df[df.isna().any(axis=1)]

Wir gehen davon aus, dass wir die Zeilen mit den fehlenden Werten sorgfältig untersucht haben. Wir betrachten die Spalte `Zufriedenheit` genauer. Mit `df.unique()` können die einzigartigen Werte eines Dataframes ausgegeben werden. 

In [ ]:
df['Zufriedenheit'].unique()

Nach sorgfältiger Überlegung entscheiden wir, dass die fehlenden Werte in dieser Spalte durch den am häufigsten vorkommenen Wert ersetzt werden sollen. `df.value_counts()` zählt wie oft welche Merkmalsausprägung vorkommt. Links steht die Merkmalsausprägung und rechts die entsprechende Anzahl.

In [ ]:
df['Zufriedenheit'].value_counts()

### Aufgabe 4: Entnehmen Sie aus der oben erhaltenen Ausgabe die Information über die häufigste Merkmalsausprägung der Zufriedenheit und speichern Sie diese in der Variable `haeufigste_zufriedenheit` ab.
Mithile von `df.fillna()` werden jetzt alle `NaNs` in der Spalte mit dem am häufigsten vorkommenen Merkmalsausprägung ersetzt. Es sind auch andere Ersetzungsarten denkbar, wie zum Beispiel dem Durchschnittswert aller Zufriedenheiten.

**Erinnerung**: Mit dem Übergabeparameter `inplace=True` wenden wir die Änderung direkt auf dem ursprünglichen Dataframe `df` an.

In [ ]:
haeufigste_zufriedenheit = 4.0
df['Zufriedenheit'].fillna(haeufigste_zufriedenheit, inplace=True)
df.shape # Ausgabe der Dimensionsgrößen des bereingten Dataframes

### Aufgabe 5: Beantworten Sie die nachfolgende Frage.

In [ ]:
%run src/02_Fragen_DAT_PJY_002.ipynb

Wir haben entschieden, dass die Durchschnittsnote ein sehr wichtiges Merkmal ist, wo es gefährlich wäre einfach Werte für die `NaNs` anzunehmen. Daher beschließen wir alle Zeilen, wo ein `NaN` bei der Durchschnittsnote steht, von der Umfrage auszunehmen und damit aus dem Datensatz zu entfernen. Hierfür nutzen wir `df.dropna()` und legen mit `subset=['Durchschnittsnote']` fest, dass nur Zeilen mit `NaNs` in der Spalte `Durchschnittsnote` entfernt werden.

In [ ]:
df.dropna(subset=['Durchschnittsnote'], inplace=True)
df.shape # Ausgabe der Dimensionsgrößen des bereingten Dataframes

Um zu überprüfen, ob alle `NaNs` ersetzt oder entfernt wurden, lassen wir uns nochmal alle Zeilen ausgeben, die ein `NaN` enthalten.

In [ ]:
df[df.isna().any(axis=1)]

Wir überprüfen weiterhin, ob es doppelte Einträgt gibt. Der nachfolgende Code extrahiert alle Zeilen aus `df`, die mindestens ein Duplikat haben. Er ist wieder in kleinere Codeabschnitt unterteilt, zur Verdeutlichung.

1. `df.duplicated(keep=False)` gibt eine boolesche Serie zurück, die angibt, ob eine Zeile ein Duplikat ist. Der Übergabeparameter `keep=False` sorgt dafür, dass alle Vorkommen von Duplikaten als True markiert werden, nicht nur die späteren oder früheren Vorkommen.

In [ ]:
df.duplicated(keep=False)

2. `df[...]`: Der DataFrame wird mit der erstellten boolschen Serie von `True` und `False` Werten gefiltert, sodass nur die Zeilen angezeigt werden, die als `True` markiert sind, also die Duplikate.

In [ ]:
df[df.duplicated(keep=False)]

Mithilfe von `df.drop_duplicates()` werden alle Zeilen aus dem Dataframe `df` entfernt, die doppelt sind. Dabei wird das erste Vorkommen der doppelten Zeile behalten mit `keep=first`.

In [ ]:
df.drop_duplicates(inplace=True, keep='first')
df.shape # Ausgabe der Dimensionsgrößen des bereingten Dataframes

### Aufgabe 6: Wie viele Zeilen enthält der Datensatz `df` jetzt noch? Begründen sie die Anzahl.

In [ ]:
df.shape # Ausgabe der Dimensionsgrößen des bereingten Dataframes

Antwort: 35 Zeilen, es wurden also 5 Zeilen entfernt. Insgesamt gab es 8 Zeilen mit `NaN`-Werten und 2 doppelte Zeilen. Von den 8 Zeilen mit fehlenden Werten waren 4 in der Spalte `Zufriedenheit`, die wir durch den häufigsten Wert ersetzt haben, also nicht entfernt haben. Die anderen 4 `NaN`-Werte waren in der Spalte `Durchschnittsnote`, wo wir die entsprechenden Zeilen gelöscht haben. Von den 2 doppelten Zeilen haben wir die letzte doppelte entfernt. Insgesamt ergibt das also 5 entfernte Zeilen.

---
## Ausreißer

Um zu identifizieren, ob es in unserem Datensatz Ausreißer gibt, wollen wir die metrischen Merkmale mithilfe eines Boxplots visualisieren.

### Aufgabe 7: Nutzen Sie die gelernten Methoden aus der *Daten: Deskriptive Statistik und Visualisierung*-Übung, um die metrischen Merkmale in Boxplots darzustellen.

In [ ]:
df.plot.box(figsize=(10, 6))

### Aufgabe 8: Welche Merkmale enthalten laut den Boxplots Ausreißer? Schauen Sie in der [Dokumentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.box.html) der Methode nach, wie diese Ausreißer definiert werden (Internetrecherche). Schauen Sie zur Interpretation der Dokumenation ggf. auch in der Vorlesung nach. 

Antwort: Alter und Semester enthalten Ausreißer. Die Position der Whisker ist standardmäßig auf 1,5*IQR (IQR = Q3 - Q1) von den Rändern der Box eingestellt. Ausreißer sind Punkte, die hinter dem Ende der Whisker liegen.

Wir wollen nun Überprüfen, ob die mithilfe der Boxplot gefundenen Ausreißer für uns wirkliche Ausreißer sind. Für das Alter sollen folgende natürliche Grenzen gelten. 
- Studierende sind maximal zwischen 10 und 100 Jahre alt

Wir filtern unseren Dataframe `df` nach diesen Kriterien und speichern die Ausreißer-Zeilen in einem neuen Dataframe `ausreisser_alter`. 

Für die Verknüpfung der Filterbedingungen nutzen wir `|`. In pandas verwendet man dieses `|` für "oder" und `&` für "und", weil diese bitweisen Operatoren elementweise auf dem Dataframe arbeiten. `or` würde nicht funktionieren, da es für einzelne boolesche Werte gedacht ist, nicht für Arrays oder Series.

In [ ]:
ausreisser_alter = df[(df['Alter'] < 10) | (df['Alter'] > 100)] # Finden der Zeilen, die außerhalb der natürlichen Grenzen des Alters liegen
ausreisser_alter

Um unseren Gesamt-Dataframe von den Ausreißern zu bereinigen können wir alle Zeilen entfernen, die außerhalb dieser natürlichen Grenzen liegen. Dafür filtern wir den Dataframe `df` direkt, um nur die Zeilen zu behalten, deren `Alter` zwischen 10 und 100 liegt. Beachten Sie den geänderten logischen bitweisen Operator (`&`)  und die geänderten Vergleichsoperatoren.

Wir nehmen an, dass die Daten selbst aufgenommen wurden und wir einen Fehler gemacht haben in der Übertragung der handschriftlichen Tabelle. Für die nachfolgenden Analysen sollen diese Daten daher nicht mit berücksichtigt und damit vom Datensatz entfernt werden. 

**Vorsicht:** Das Entfernen von Datenpunkten nur weil sie nicht zur Hypothese o.ä. passen ist wissenschaftlich schlechtes Verhalten und sollte unter keinen Umständen passieren. Ein Entfernen von Datenpunkten muss immer gut begründet und dokumentiert sein. Außerdem gibt es auch noch andere Möglichkeiten mit Ausreißern umzugehen. 

In [ ]:
df = df[(df['Alter'] >= 10) & (df['Alter'] <= 100)]

In [ ]:
df.shape

### Aufgabe 9: Überlegen Sie sich sinnvolle natürliche Grenzen für das Merkmal `Semester`. Übertragen Sie diese dann in den untenstehenden Code und lassen Sie sich die Zeilen ausgeben, die außerhalb dieser Grenzen liegen. Entfernen Sie die Zeilen aber **nicht** aus `df`.

In [ ]:
ausreisser_semester = df[(df['Semester'] < 1) | (df['Semester'] > 50)]
ausreisser_semester

Wir bemerken, dass wir damals geschmiert haben und der Strich vor der `-7` bei der Semesteranzahl nur durch ein Ausrutschen zustande kam. Daher entscheiden wir, den Wert in `7` zu ändern. Mithilfe von `df.index` wird der Index der Zeile(n) eines Dataframes ausgegeben. Dieser befindet sich in unserem Beispiel noch vor der Spalte `ID`. 

In [ ]:
ausreisser_semester_index = ausreisser_semester.index
ausreisser_semester_index

Basierend auf diesem Index und dem Spaltennamen können wir jetzt mithilfe von `df.loc[index, spaltenname]` gezielt auf einzelne Merkmalsausprägungen zugreifen.

In [ ]:
df.loc[ausreisser_semester_index, 'Semester']

Bei der Ausgabe sehen wir als ersten Wert den Index der Zeile und als zweiten Wert den Wert der Merkmalsausprägung in der ausgewählten Spalte (`Name: Semester`). 

Nun überschreiben wir diesen Wert mit einer 7.

In [ ]:
df.loc[ausreisser_semester_index, 'Semester'] = 7

Zur Überprüfung schauen wir uns die beiden bereinigten Spalten nochmal in einem Boxplot an.

In [ ]:
df[['Alter', 'Semester']].plot.box()

Wir erkennen, dass laut dem Boxplot immer noch ein Ausreißer in der Spalte `Alter` enthalten ist. Da es aber durchaus möglich sein kann, dass jemand mit 67 studiert und diese Person daher in unseren natürlichen Grenzen liegt, sind wir zufrieden mit dem Ergebnis.

---
### Konsistenz in Format und Sprache

Wir wollen uns einmal die einzelnen einzigartigen Werte der Spalte `Geschlecht` ausgeben lassen.

In [ ]:
df['Geschlecht'].unique()

Es gibt mehrere Merkmalsausprägungen, die für das gleiche stehen. Zum Beispiel stehen `w` und `f` beide für `weiblich`.

### Aufgabe 10 Beantworten Sie die nachfolgende Frage.

In [ ]:
%run src/03_Fragen_DAT_PJY_002.ipynb

Zunächst werden wir die falschen Einträge korrigieren, indem wir die Werte durch die zusammengefassten Merkmalsausprägungen ersetzen. Außerdem wollen wir, dass männliche Studenten anstelle von `m` die Merkmalsausprägung `maennlich` haben. 

Dazu verwenden wir die Methode `df.replace()`.

In [ ]:
df['Geschlecht'].replace({'w':'weiblich', 'f':'weiblich', 'm':'maennlich'}, inplace=True)

Die Methode `df.replace()` erwartet ein **dictionary**. Ein **dictionary** ist innerhalb von `{}` definiert und besteht aus Schlüssel-Wert-Paaren. Zuerst wird der Schlüssel angegeben, gefolgt von einem `:` und dem Wert. Im obigen Code geben wir an, dass wir die Methode `df.replace()` auf die Spalte `Geschlecht` (`df['Geschlecht'])`) anwenden wollen. Als Wert geben wir ein **dictionary** an, das als Schlüssel die zu ersetzenden Werte und als Wert die neuen Werte enthält. Wir geben also an, dass `w` durch `weiblich`, `f` durch `weiblich` und `m` durch `maennlich` ersetzt werden soll. Die einzelnen Schlüssel-Wert-Paare werden durch Kommata getrennt.

Wir kontrollieren nochmal das Ergebnis:

In [ ]:
df['Geschlecht'].unique()

Der Datentyp der Spalte ist `object`. Es gibt einen geeigneteren Datentyp `category` in den wir diesen nominalen Datentyp mithilfe von `df.astype()` umwandeln wollen.

In [ ]:
df['Geschlecht'] = df['Geschlecht'].astype('category')

Der Datentyp `category` ist vor allem nützlich da er gegenüber `object` Speicherplatz spart. Außerdem ist er hilfreich, wenn die lexikalische Reihenfolge einer Variable nicht der logischen Reihenfolge entspricht (z.B. "eins", "zwei", "drei"), kann durch die Umwandlung in eine kategorische Variable und das Festlegen einer Reihenfolge die logische statt der lexikalischen Reihenfolge verwendet werden.
Außerdem wissen so andere Python-Bibliotheken, dass diese Spalte als kategorische Variable behandelt werden soll, was für geeignete statistische Methoden oder Visualisierungen nützlich ist.

Wir wollen das Ergebnis überprüfen.

In [ ]:
df.info()

Uns fällt auf, dass die Spalte `Hat_Nebenjob` vom Typ `int64` ist. Wir überprüfen noch einmal die einzigartigen Merkmalsausprägungen. 

In [ ]:
df['Hat_Nebenjob'].unique()

Innerhalb des Datentyps `bool` gibt es nur die beiden Merkmalsausprägungen `True` und `False`. Daher ist er speichereffizienter als z.B. `int64`. Da er außerdem noch eindeutiger, leistungsoptimierter und kompatibler ist, sollte er hier bevorzugt verwendet werden. 

### Aufgabe 11: Wandeln Sie den Datentyp der Spalte `Hat_Nebenjob` in `bool` um und überprüfen Sie nochmal das Ergebnis.

In [ ]:
df['Hat_Nebenjob'] = df['Hat_Nebenjob'].astype('bool')

In [ ]:
df['Hat_Nebenjob'].unique()

---
## One Hot Encoding
Viele Algorithmen für maschinelles Lernen können nicht mit mehrwertigen kategorialen Attributen umgehen. Um daher weitere Analysen mit dem ursprünglichen Datensatz machen zu können, müssen die vorliegenden kategorischen Merkmale binär codiert werden mithilfe von `pd.get_dummies()`. Mit `columns=['Geschlecht','Abschluss','Studiengang']` geben wir an, für welche Spalten das One Hot Encoding durchgeführt werden soll. Beobachten Sie, wie sich die Spaltenanzahl verändert.

In [ ]:
df = pd.get_dummies(df, columns=['Geschlecht','Abschluss','Studiengang'])
df

---
## Anwendung auf einen neuen Datensatz
Die oben (und im letzten Praktikum) kennengelernten Methoden zur Datenvorverarbeitung sollen nun auf dem neuen Datensatz `movies.csv` angewandt werden. Der Datensatz enthält viele Informationen über Filme und soll von Ihnen vorverarbeitet werden.

### Aufgabe 12: Importieren Sie den Datensatz `movies.csv`.

In [ ]:
df_movies = pd.read_csv('./data/movies.csv')

### Aufgabe 13: Verschaffen Sie sich eine Überblick über den Datensatz und beantworten Sie dann die nachfolgenden Fragen.

In [ ]:
df_movies.head()

In [ ]:
df_movies.info()

In [ ]:
print('IMDb-Bewertung Minimum: %f' % df_movies['IMDB-Bewertung'].min())
print('IMDb-Bewertung Maximum: %f' % df_movies['IMDB-Bewertung'].max())
print('Rotten-Tomatoes-Bewertung Minimum: %f' % df_movies['Rotten-Tomatoes-Bewertung'].min())
print('Rotten-Tomatoes-Bewertung Maximum: %f' % df_movies['Rotten-Tomatoes-Bewertung'].max())

In [ ]:
# Anzahl fehlender Werte pro Spalte anzeigen
print("Fehlende Werte pro Spalte:")
print(df_movies.isna().sum())

In [ ]:
%run src/04_Fragen_DAT_PJY_002.ipynb

### Aufgabe 14: Recherchieren Sie im Internet was die `IMDB-Bewertung` und die `Rotten-Tomatoes-Bewertung` bedeuten und geben Sie für beide die möglichen Bewertungsbereiche an (z.B. sowas wie 1-6 bei Schulnoten). Decken sich die Bewertungsbereiche mit Ihren gefundenen min- und max-Werten?

Antwort: Die IMDb-Bewertung und die Rotten-Tomatoes-Bewertung sind zwei unterschiedliche Systeme zur Bewertung von Filmen und Fernsehsendungen:

### IMDb-Bewertung:
- **Bedeutung**: Die IMDb-Bewertung ist eine numerische Bewertung, die von den Nutzern der Internet Movie Database (IMDb) abgegeben wird. Sie reflektiert die durchschnittliche Bewertung, die ein Film oder eine Serie von den Nutzern erhalten hat.
- **Bewertungsbereich**: 1 bis 10. Nutzer können Filme und Serien auf einer Skala von 1 (schlecht) bis 10 (exzellent) bewerten.

### Rotten-Tomatoes-Bewertung:
- **Bedeutung**: Rotten Tomatoes bietet zwei Hauptbewertungen: den Tomatometer-Score und die Audience Score. Der Tomatometer-Score basiert auf den Bewertungen von professionellen Filmkritikern, während der Audience Score auf den Bewertungen von normalen Zuschauern basiert.
- **Bewertungsbereich**: 0% bis 100%. Der Tomatometer-Score zeigt den Prozentsatz der positiven Kritiken an, die ein Film oder eine Serie erhalten hat.

### Vergleich mit den gefundenen Werten:
Die gefundenen Werte von 1-9 für IMDb und 1-100 für Rotten Tomatoes decken sich nicht vollständig mit den tatsächlichen Bewertungsbereichen, also sind in den Filmen in der Tabelle nicht alle möglichen Merkmalsausprägungen vertreten. Der IMDb-Bereich sollte 1-10 sein, während der Rotten-Tomatoes-Bereich korrekt mit 0-100 angegeben ist.geben ist.0-100 angegeben ist.

### Aufgabe 15: Behandeln Sie eventuelle Duplikate und Leerstellen und argumentieren Sie jeweils, wieso Sie etwas gemacht haben. Geben Sie für jeden Schritt an, wie viele Datensätze danach noch im bereinigten Dataframe verbleiben und warum.

In [ ]:
df_movies[df_movies.isna().any(axis=1)] # Anzeigen lassen der Zeilen mit NaNs

In [ ]:
# Neue Kategorie für fehlende Hauptgenre-Werte, da die Spalte nicht so relevant für uns ist und wir die Zeilen nicht entfernen wollen, aber auch nichts annehmen wollen dafür
df_movies['Hauptgenre'].fillna('Undefined', inplace=True)
print(df_movies.isna().sum())
df_movies.shape # Es verbleiben weiterhin 3205 Zeilen, da nur Werte ersetzt, aber keine Zeilen entfernt wurden

In [ ]:
# Entfernen von Zeilen mit fehlenden Werten in wichtigen Spalten, wo wir keinen Wert annehmen wollen
df_movies.dropna(subset=['Weltweiter_Einspielumsatz', 'Produktionskosten', 'IMDB-Bewertung', 'Rotten-Tomatoes-Bewertung', 'Titel'], inplace=True)
print(df_movies.isna().sum())
df_movies.shape # Es verbleiben 2256 Zeilen, die verbleibenden Zeilen wurden entfernt, 
                # da wir für die entsprechenden Spalten keinen Wert annehmen wollten

In [ ]:
# Da die Laufzeit für die Analysen, die nachfolgenden gemacht werden sollen, nicht von großer Relevanz ist aber viele Zeilen betrifft, 
# nehmen wir hierfür den Mittelwert aller Filme an.
mean_laufzeit = df_movies['Laufzeit_min'].mean()
print(mean_laufzeit)
df_movies['Laufzeit_min'].fillna(mean_laufzeit, inplace=True)
print(df_movies.isna().sum())
df_movies.shape # Es verbleiben 2256 Zeilen, da nur Werte mit dem Mittelwert für die Laufzeit ersetzt und keine Zeilen entfernt wurden

In [ ]:
# Überprüfen der Größe des bereinigten Dataframes
df_movies.shape

In [ ]:
df_movies[df_movies.duplicated(keep=False)]

In [ ]:
# Entfernen von Duplikaten
df_movies.drop_duplicates(inplace=True, keep='first')
print(df_movies.shape) # Es verbleiben 2254 Zeilen, da die 2 Zeilen nach dem ersten Auftauchen der ersten Zeile entfernt wurden
df_movies[df_movies.duplicated(keep=False)] # Überprüfen des Ergebnis

In [ ]:
%run src/05_Fragen_DAT_PJY_002.ipynb

### Aufgabe 16: Bringen Sie die Spalte `Hauptgenre` in ein einheitliches und konsistentes Format und passen Sie den Datentyp an.

In [ ]:
# Analyse der Genre-Kategorien
print("Einzigartige Hauptgenre:")
df_movies['Hauptgenre'].unique()

In [ ]:
df_movies['Hauptgenre'].replace({'Docu':'Documentary', 'Document.':'Documentary'}, inplace=True)

In [ ]:
df_movies['Hauptgenre'].unique() # Überprüfen des Ergebnis

In [ ]:
df_movies['Hauptgenre'] = df_movies['Hauptgenre'].astype('category')
df_movies.info()

### Aufgabe 17: Definieren, visualisieren und behandeln Sie eventuelle Ausreißer in `Laufzeit_min`. Argumentieren Sie wieder wieso Sie welche Methode angewandt haben. Beantworten Sie auch die Frage im Anschluss.

In [ ]:
plt.close('all')
df_movies['Laufzeit_min'].plot.box()
plt.show() # Der Plot ist daher so verzerrt und man erkennt keinen richtigen Boxplot mehr, da wir 1994 Werte(über die Hälfte)
           # mit dem Mittelwert ersetzt haben. 

In [ ]:
# Natürliche Grenzen für Laufzeit
ausreisser_laufzeit = df_movies[(df_movies['Laufzeit_min'] < 0)] # Wir lassen grundsätzliche alle Laufzeiten zu, da es Kurzfilme und
                                                                 # und sehr lange Filme gibt, aber negative Laufzeiten gibt es nicht.
ausreisser_laufzeit

In [ ]:
# Wir gehen davon aus/wir wissen, dass es sich um ein Vorzeichenfehler handelt, daher ersetzen wir die Werte mit dem Betrag abs()
df_movies['Laufzeit_min'] = abs(df_movies['Laufzeit_min'])

In [ ]:
%run src/06_Fragen_DAT_PJY_002.ipynb

### Aufgabe 18: Führen Sie das One Hot Encoding für entsprechende(n) Spalte(n) durch. 

In [ ]:
# One Hot Encoding für kategorische Variablen
df_movies = pd.get_dummies(df_movies, columns=['Hauptgenre'])

In [ ]:
df_movies.head()

### Bonusaufgabe: Return on Investment (ROI) Analyse

Finden Sie die profitabelsten Filme basierend auf dem Return on Investment (ROI). Der ROI berechnet sich als:
```
ROI = (Weltweiter_Einspielumsatz - Produktionskosten) / Produktionskosten * 100
```

Sie können einem Dataframe eine neue Spalte hinzufügen, indem Sie z.B `df_movies['neueSpalte'] = 1` ausführen. Jetzt würde in der Spalte `neueSpalte` in jeder Zeile eine 1 stehen.

Um den Datensatz zu sortieren anhand der angegebenen Spalte (`by='Spaltenname'`) kann die Methode `df.sort_values(by='Spaltenname', ascending=False)` genutzt werden. Standardmäßig ist `ascending=True`, sodass die kleinsten Werte oben stehen und die größten unten.

In [ ]:
# ROI Berechnung
df_movies['ROI'] = (df_movies['Weltweiter_Einspielumsatz'] - df_movies['Produktionskosten']) / df_movies['Produktionskosten'] * 100
df_movies.sort_values(by='ROI', ascending=False)

### Bonusaufgabe: Jemand behauptet, je mehr Geld man in einen Film steckt, also je höher die Produktionskosten sind, desto besser ist auch die IMDB-Bewertung. Stimmt das? Visualisieren Sie und geben Sie außerdem einen Wert aus, der den Zusammenhang quantifizieren könnte. Stimmt die Aussage?

**Hinweis**: Schauen Sie zur Hilfe in der *Daten: Deskriptive Statistik und Visualisierung*-Übung nach.

In [ ]:
plt.close('all')
sns.scatterplot(data=df_movies, x='Produktionskosten', y='IMDB-Bewertung')

In [ ]:
df_movies.corr(method='pearson', numeric_only=True)

Antwort: Die Korrelation beträgt ungefähr 0.016, es gibt also danach so gut wie keinen Zusammenhang zwischen den Merkmalen. Im Plot sieht man, dass die Chance eine schlechte Bewertung zu bekommen mit dem Steigen der Produktionskosten sinkt, es aber auch viele gute Bewertungen bei sehr geringen Produktionskosten gibt. 

### Bonusaufgabe: Stimmt es, dass je mehr Geld man in die Produktion steckt, desto mehr Geld wird dann auch weltweit eingespielt?

In [ ]:
plt.close('all')
sns.scatterplot(data=df_movies, x='Produktionskosten', y='Weltweiter_Einspielumsatz')

Antwort: Die Korrelation beträgt ungefähr 0.66, damit kann man schon von einem positiven Zusammenhang der beiden Merkmale ausgehen. Auch im Plot sieht man, dass die Chance schlechte Einspielzahlen zu erreichen mit dem Steigen der Produktionskosten sinkt und dieEinspielzahlen mit höheren Produktionskosten steigen.

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Die Übungsserie begleitend zum AI4ALL-Kurs</span> der <span xmlns:cc="http://creativecommons.org/ns#" property="cc:attributionName">EAH Jena</span> ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Namensnennung - Nicht kommerziell - Keine Bearbeitungen 4.0 International Lizenz</a>.

Der AI4ALL-Kurs entsteht im Rahmen des Projekts MoVeKI2EAH. Das Projekt MoVeKI2EAH wird durch das BMBF (Bundesministerium für Bildung und Forschung) und den Freistaat Thüringen im Rahmen der Bund-Länder-Initiative zur Förderung von Künstlicher Intelligenz in der Hochschulbildung gefördert (12/2021 bis 11/2025, Föderkennzeichen 16DHBKI081).